In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import csv, datetime, random
import matplotlib.dates as mdates

In [ ]:
def MakeDataFromText(filepath):
    """
    Helper function to parse text files in IFM proprietary format.
    """
    with open(filepath, 'r') as file:
        reader = csv.reader(file)

        db = dict()                          # Dictionary that will hold k,v pairs of 
        recent_EPC = ''                      # The most recent EPC that was read, and data is being assigned to
        Time_Or_Data = True                  # Time is true, Data is false
        reached_24 = False

        for l in reader:                     # Iterate over entire text file
            line = l[0]                      # Each line only has one token, so we select it
            
            if len(line)==24:                # If it is an EPC
                db[line] = [[],[]]           # Create new k,v pair in database
                recent_EPC = line            # Current EPC to add to is selected
            
            elif line == '[':                # If start bracket, beginning to collect an array of data
                data = []

            elif line == ']':                # End bracket means end of data array
                if len(data)!=0:
                    if Time_Or_Data:
                        db[recent_EPC][0] = data
                        Time_Or_Data = False
                    else:
                        db[recent_EPC][1] = data
                        Time_Or_Data = True
                data = []

            elif len(line)==8:               # If it is a time stamp
                h,m,s = line.split(':')      # Parse timestamp into hours, minutes, seconds

                if int(h)==23:
                    reached_24 = True        # If end of day reached, must move to Day 2

                if reached_24 and int(h) < 12:
                    day=2
                else:
                    day=1

                data.append(datetime.datetime(1970, 2, day, int(h) - 3, int(m), int(s))) # Minus 3 for Pacific Time
            
            elif line ==' ':               # If blank space, then we skip to parse next line
                pass

            else:                          # If it is a data point
                try:
                    data.append(float(line))
                except:
                    pass                   # Error catching case just in case, so annoying errors don't crash program
    return db

class Person:
    """
    Class to represent each individal poutfit being recorded during Deployment, made up of garments.
    """
    def __init__(
        self, wet:bool, Name:str, color:str,
        backneck:str,   back:str,        chest:str, 
        leftab:str,     rightab:str, 
        rightuparm:str, rightlowarm:str, 
        leftuparm:str,  leftlowarm:str, 
        beanie1:str,    beanie2:str,     beanie3:str, beanieR:str, beanieL:str
    ):
        self.Backneck    = backneck        # Tag ID's for the shirt
        self.Back        = back
        self.Chest       = chest
        self.Leftab      = leftab
        self.Rightab     = rightab
        self.Rightuparm  = rightuparm
        self.Rightlowarm = rightlowarm
        self.Leftuparm   = leftuparm
        self.Leftlowarm  = leftlowarm
        self.Beanie1     = beanie1         # Tag ID's for the beanies
        self.Beanie2     = beanie2
        self.Beanie3     = beanie3
        self.BeanieR     = beanieR
        self.BeanieL     = beanieL

        self.Name = Name                   # Soldier's Name

        self.ShirtMapping = {
            self.Backneck:    Name + " Back Neck",
            self.Back:        Name + " Back",
            self.Chest:       Name + " Chest",
            self.Leftab:      Name + " Left Abdomen",
            self.Rightab:     Name + " Right Abdomen",
            self.Rightuparm:  Name + " Right Upper Arm",
            self.Rightlowarm: Name + " Right Lower Arm",
            self.Leftlowarm:  Name + " Left Lower Arm",
            self.Leftuparm:   Name + " Left Upper Arm",
        }

        self.TagList = [
            self.Backneck,   self.Back, self.Chest,
            self.Leftab,     self.Rightab,
            self.Rightuparm, self.Rightlowarm,
            self.Leftuparm,  self.Leftlowarm,
            self.Beanie1,    self.Beanie2, self.Beanie3, self.BeanieR, self.BeanieL
        ]

        self.BeanieList = [self.Beanie1, self.Beanie2, self.Beanie3, self.BeanieR, self.BeanieL]
        self.Color = color

        self.wet = wet # Bool for wetness check
        if wet:
            self.BackColor = "#870101"
            self.BackNeckColor = "#ad4503"
            self.ChestColor = "#756a01"
            self.LeftAbColor = "#396102"
            self.RightAbColor = "#015c0b"
            self.LeftUpArmColor = "#01663f"
            self.RightUpArmColor = "#00a191"
            self.LeftLowArmColor = "#012173"
            self.RightLowArmColor = "#1e075e"
            
        else:
            self.BackColor = "#ff0d0d"
            self.BackNeckColor = "#ff7214"
            self.ChestColor = "#d1bc02"
            self.LeftAbColor = "#6bb803"
            self.RightAbColor = "#01bf16"
            self.LeftUpArmColor = "#03c278"
            self.RightUpArmColor = "#06b4cf"
            self.LeftLowArmColor = "#0443e0"
            self.RightLowArmColor = "#4c05e6"

In [ ]:
# Paul P, only wet
person6  = Person(True, "Paul P.", "#de0413", "71BB", "7705", "B25E", "3247", "A9B5", "6C38", "7662", "A983", "098F", "B644", "7BCC", "B576", "70D1", "4D84");

# Joelle, only wet
person14 = Person(True, "Joelle", "#9e05a1", "462C", "5B60", "7415", "6310", "1851", "616A", "5DAA", "6D28", "9991", "3B0E", "5374", "682C", "0A50", "1543");

# Garrett C, only wet
person28 = Person(True, "Garrett C.", "#0909eb", "8977", "4EB3", "AA5E", "6626", "9ABE", "6CD0", "AB8E", "14AA", "6E94", "895D", "571F", "2D68", "8121", "198C");

# Dennis, only wet
person2  = Person(True, "Dennis", "#078c26", "333B", "289B", "2473", "231D", "9879", "4067", "5FB6", "169E", "8D50", "74C4", "73DC", "DC4A", "884D", "1BA4");

# Brent, only wet
person17 = Person(True, "Brent L.", "#e8c100", "6CC7", "8FB7", "799D", "6F8D", "6332", "5A1E", "92A5", "4A3D", "3EC9", "16AB", "A66C", "6162", "A686", "7E1B");

# Jackson, wet and dry
person15 = Person(True,  "Jackson Wet", "#0c008f", "9C54", "9968", "65E1", "5E23", "ACB3", "7AD3", "99A1", "B19D", "43C6", "7DCB", "9D35", "6FD0", "34CE", "62E3");
person11 = Person(False, "Jackson Dry", "#3845ff", "2A1B", "238C", "731F", "9591", "5C98", "5F06", "4526", "461C", "5253", "B078", "3AC7", "566B", "65D4", "432B");

# Matt C, wet and dry
person16 = Person(True,  "Matt C. Wet", "#02591e", "959E", "1C5F", "5A59", "077A", "902A", "3B60", "8199", "4469", "813A", "B7A4", "5163", "4DA5", "8995", "7972");
person8  = Person(False, "Matt C. Dry", "#00ed4f", "1B6A", "0D42", "7AD4", "20AF", "493F", "404A", "6878", "1A3B", "546F", "194B", "3133", "A847", "0D50", "7A61");

# William D, wet and dry
person13 = Person(True,  "William D. Wet", "#a60000", "A033", "A0C0", "892F", "627F", "6BD9", "3DB0", "8C97", "4ECF", "A73C", "90AE", "78D3", "82D8", "312D", "2A29");
person10 = Person(False, "William D. Dry", "#ff3037", "366C", "A08B", "AC42", "9AC9", "B53F", "76A4", "5E76", "68AE", "41D6", "803F", "5757", "9E54", "----", "----");

# Sean, wet and dry
person32 = Person(True,  "Sean Wet", "#7d0080", "2990", "1EC3", "51E2", "3783", "AE5E", "4575", "0688", "A0B0", "1BA6", "3C3E", "1989", "7C2C", "42D7", "6129");
person31 = Person(False, "Sean Dry", "#fb24ff", "1CAE", "7764", "8F24", "AD3E", "8898", "413F", "097A", "18A8", "4258", "7F4E", "801E", "8651", "AAB4", "6CB5");

# Harold, wet and dry
person19 = Person(True,  "Harold Wet", "#003952", "697F", "78A5", "5D0E", "7EC6", "AE59", "8158", "4A9B", "1D44", "2122", "5C02", "2D90", "6DAB", "3374", "B1AB");
person21 = Person(False, "Harold Dry", "#008dc9", "8CB4", "4115", "4D74", "4883", "4C9F", "ABB7", "7938", "390F", "21A5", "A697", "7C7D", "B689", "114D", "047A");

# Katherina, wet and dry
person26 = Person(True,  "Katherina Wet", "#965500", "4813", "2791", "1F7A", "6F4A", "2383", "2F66", "2E5C", "926A", "2E56", "158F", "5B06", "B28A", "73C6", "819C");
person22 = Person(False, "Katherina Dry", "#ed8600", "4F0D", "90D5", "0874", "45CE", "6CAC", "3EB2", "A26F", "888C", "50C3", "3767", "664F", "36A4", "2759", "BB8B");

b_gym1 = MakeDataFromText("MWTC/baseline_gym_1.txt")
b_gym2 = MakeDataFromText("MWTC/baseline_gym_2.txt")
b_gym3 = MakeDataFromText("MWTC/baseline_gym_3.txt")
b_gym4 = MakeDataFromText("MWTC/baseline_gym_4.txt")

In [ ]:
Back_count = 0
neck_count = 0
larm_count = 0
rarm_count = 0
low_count = 0
row_count = 0
lab_count = 0
rab_count = 0
chst_count = 0
b1_count = 0
b2_count = 0
b3_count = 0
bL_count = 0
bR_count = 0
Back_sum = 0
neck_sum = 0
larm_sum= 0
rarm_sum= 0
low_sum = 0
row_sum = 0
lab_sum = 0
rab_sum = 0
chst_sum = 0
b1_sum = 0
b2_sum = 0
b3_sum = 0
bL_sum = 0
bR_sum = 0

p = person17

for d in [b_gym1, b_gym2, b_gym3, b_gym4]:
    for k,v in d.items():
        id = k[-4:]

        for reading in v[1]:
                    if id==p.Back:
                        Back_sum += reading
                        Back_count += 1
                    elif id==p.Backneck:
                        neck_sum += reading
                        neck_count += 1
                    elif id==p.Leftuparm:
                        larm_sum += reading
                        larm_count+=1
                    elif id==p.Rightuparm:
                        rarm_sum += reading
                        rarm_count+=1
                    elif id==p.Leftlowarm:
                        low_sum += reading
                        low_count +=1
                    elif id==p.Rightlowarm:
                        row_sum += reading
                        row_count+=1
                    elif id==p.Leftab:
                        lab_sum += reading
                        lab_count+=1
                    elif id==p.Rightab:
                        rab_sum += reading
                        rab_count +=1
                    elif id==p.Chest:
                        chst_sum += reading
                        chst_count +=1
                    elif id==p.Beanie1:
                        b1_sum += reading
                        b1_count +=1
                    elif id==p.Beanie2:
                        b2_sum += reading
                        b2_count +=1
                    elif id==p.Beanie3:
                        b3_sum += reading
                        b3_count +=1
                    elif id==p.BeanieL:
                        bL_sum += reading
                        bL_count +=1
                    elif id==p.BeanieR:
                        bR_sum += reading
                        bR_count +=1

print("Back: ", round(Back_sum/Back_count,2)  , Back_count)
print("neck: ", round(neck_sum/neck_count,2)  , neck_count)

In [ ]:
print("larm: ", round(larm_sum/larm_count,2)  , larm_count)
print("low: ", round(low_sum/low_count,2)  , low_count)
print("rarm: ", round(rarm_sum/rarm_count,2)  , rarm_count)
print("row: ", round(row_sum/row_count,2)  , row_count)

In [ ]:
print("lab: ", round(lab_sum/lab_count,2)  , lab_count)
print("rab: ",round( rab_sum/rab_count,2)  , rab_count)
print("chst: ", round(chst_sum/chst_count,2)  , chst_count)

In [ ]:
print("b1: ", round(b1_sum/b1_count,2)  , b1_count)
print("b2: ", round(b2_sum/b2_count,2)  , b2_count)
print("b3: ", round(b3_sum/b3_count,2)  , b3_count)

In [ ]:
print("bL: ", round(bL_sum/bL_count,2)  , bL_count)
print("bR: ", round(bR_sum/bR_count,2)  , bR_count)